[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-1/deployment.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239303-lesson-8-deployment)


# 部署（Deployment）

## 回顾

我们已经构建了一个带记忆的智能体：

* `act` —— 让模型调用特定工具
* `observe` —— 将工具输出传回给模型
* `reason` —— 让模型基于工具输出进行推理，以决定下一步（调用工具或直接回复）
* `persist state` —— 使用内存检查点器，支持带中断的长对话

## 目标

现在，我们将介绍如何把智能体实际部署到本地的 Studio，以及部署到 `LangGraph Cloud`。


In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph_sdk langchain_core

## 概念

需要理解的几个核心概念：

`LangGraph` ——
- Python 与 JavaScript 库
- 用于创建智能体工作流

`LangGraph API` ——
- 打包图代码
- 提供任务队列以管理异步操作
- 为跨交互维持状态提供持久化能力

`LangGraph Cloud` ——
- LangGraph API 的托管服务
- 支持从 GitHub 仓库部署图
- 提供已部署图的监控与追踪
- 每个部署都有唯一 URL 可访问

`LangGraph Studio` ——
- LangGraph 应用的集成开发环境（IDE）
- 以 API 为后端，支持在界面中实时测试与探索图
- 可本地运行或配合云部署

`LangGraph SDK` ——
- 用于以编程方式与 LangGraph 图交互的 Python 库
- 无论本地还是云端，都提供一致的操作接口
- 支持创建客户端、访问助理、管理线程、执行运行等


In [2]:
if "google.colab" in str(get_ipython()):
    raise Exception(
        "Unfortunately LangGraph Studio is currently not supported on Google Colab"
    )

In [3]:
from langgraph_sdk import get_client

In [4]:
# This is the URL of the local development server
URL = "http://127.0.0.1:2024"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [5]:
assistants[-3]

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'config': {},
 'context': {},
 'metadata': {'created_by': 'system'},
 'name': 'agent',
 'created_at': '2025-09-19T16:00:03.733457+00:00',
 'updated_at': '2025-09-19T16:00:03.733457+00:00',
 'version': 1,
 'description': None}

In [6]:
# We create a thread for tracking the state of our run
thread = await client.threads.create()

## 本地测试

**⚠️ 说明**

自这些视频录制以来，我们已更新了 Studio，使其可以在本地运行并在浏览器中打开。现在推荐以这种方式运行 Studio（而不是视频中展示的桌面应用）。有关本地开发服务器的文档见[这里](https://langchain-ai.github.io/langgraph/concepts/langgraph_studio/#local-development-server)，如何运行本地 Studio 见[这里](https://langchain-ai.github.io/langgraph/how-tos/local-studio/#run-the-development-server)。要启动本地开发服务器，请在本模块的 `/studio` 目录下运行：

```
langgraph dev
```

你应该能看到如下输出：
```
- 🚀 API: http://127.0.0.1:2024
- 🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
- 📚 API Docs: http://127.0.0.1:2024/docs
```

在浏览器中打开 Studio UI：`https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024`。


In [7]:
from langchain_core.messages import HumanMessage

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
    thread["thread_id"],
    "agent",
    input=input,
    stream_mode="values",
):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data["messages"][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '8ff20328-5fea-4fb9-b70f-b3bce5f76a9a', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_4f73c642988443f4939c49', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 3, "b": 2}'}}]}, 'response_metadata': {'model_name': 'qwen-plus', 'finish_reason': 'tool_calls', 'request_id': '66a2333b-5e0b-4ccc-a27b-bf4e377ff1b1', 'token_usage': {'input_tokens': 352, 'output_tokens': 24, 'total_tokens': 376, 'prompt_tokens_details': {'cached_tokens': 0}}}, 'type': 'ai', 'name': None, 'id': 'run--67047a11-1288-44c5-9a1a-e28af4f5dcee-0', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_4f73c642988443f4939c49', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multi

## 云端测试

我们可以通过 LangSmith 部署到云端，步骤见[这里](https://langchain-ai.github.io/langgraph/cloud/quick_start/#deploy-from-github-with-langgraph-cloud)。

### 在 GitHub 创建新仓库

* 打开你的 GitHub 账户
* 点击右上角的 “+” 并选择 `"New repository"`
* 为仓库命名（例如 `langchain-academy`）

### 将你的 GitHub 仓库添加为远程

* 回到最开始克隆 `langchain-academy` 的本地终端
* 把刚创建的 GitHub 仓库添加为远程

```
git remote add origin https://github.com/your-username/your-repo-name.git
```
* 推送到远程
```
git push -u origin main
```

### 将 LangSmith 连接到你的 GitHub 仓库

* 打开 [LangSmith](hhttps://smith.langchain.com/)
* 点击左侧的 `deployments` 选项卡
* 选择 `+ New Deployment`
* 选择刚才创建的 GitHub 仓库（例如 `langchain-academy`）
* 将 `LangGraph API config file` 指向其中一个 `studio` 目录
* 例如，对 module-1 使用：`module-1/studio/langgraph.json`
* 设置你的 API keys（例如，可以直接从 `module-1/studio/.env` 文件复制）

![Screenshot 2024-09-03 at 11.35.12 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fd61c93d48e5d0f47_deployment2.png)

### 与部署交互

我们可以通过几种方式与部署交互：

* 使用 [SDK](https://langchain-ai.github.io/langgraph/cloud/quick_start/#use-with-the-sdk)，同之前一样；
* 使用 [LangGraph Studio](https://langchain-ai.github.io/langgraph/cloud/quick_start/#interact-with-your-deployment-via-langgraph-studio)。

![Screenshot 2024-08-23 at 10.59.36 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fa159a09a51d601de_deployment3.png)

若要在本笔记本中直接使用 SDK，请确保已设置 `LANGSMITH_API_KEY`！


In [ ]:
import os, getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("LANGSMITH_API_KEY")

In [ ]:
# Replace this with the URL of your deployed graph
URL = "https://langchain-academy-8011c561878d50b1883f7ed11b32d720.default.us.langgraph.app"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [37]:
# Select the agent
agent = assistants[0]

In [38]:
agent

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'created_at': '2024-08-23T17:58:02.722920+00:00',
 'updated_at': '2024-08-23T17:58:02.722920+00:00',
 'config': {},
 'metadata': {'created_by': 'system'}}

In [ ]:
from langchain_core.messages import HumanMessage

# We create a thread for tracking the state of our run
thread = await client.threads.create()

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
    thread["thread_id"],
    "agent",
    input=input,
    stream_mode="values",
):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data["messages"][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '8ea04559-f7d4-4c82-89d9-c60fb0502f21', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27'}, 'type': 'ai', 'name': None, 'id': 'run-b0ea5ddd-e9ba-4242-bb8c-80eb52466c76', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '1bf558e7-79ef-4f21-bb66-acafbd04677a', 'tool_call_id': 'c